In [1]:
import torch
from torch.utils.data import IterableDataset, DataLoader, get_worker_info

In [3]:
from multiprocessing.managers import DictProxy
from torch.multiprocessing import Manager
from torch.multiprocessing.spawn import spawn

class SharedDictDataset(IterableDataset):
    def __init__(self, shared_dict: DictProxy):
        self.shared_dict = shared_dict

    def __iter__(self):
        worker_info = get_worker_info()
        self.shared_dict[0] = worker_info.id
        yield from self.shared_dict.items()


def main():
    print(get_worker_info())
    m = Manager()
    d = {i: i+1 for i in range(100_000_000)}
    shared_dict = d
    dataset = SharedDictDataset(shared_dict)
    dataloader = DataLoader(dataset, num_workers=4)
    for item in dataloader:
        print(item)
    del m


spawn(main, nprocs=4, join=True)


Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/lfs/ampere9/0/roed/env/micromamba/envs/dogma-data/lib/python3.11/multiprocessing/spawn.py", line 122, in spawn_main
    exitcode = _main(fd, parent_sentinel)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/lfs/ampere9/0/roed/env/micromamba/envs/dogma-data/lib/python3.11/multiprocessing/spawn.py", line 132, in _main
    self = reduction.pickle.load(from_parent)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: Can't get attribute 'main' on <module '__main__' (built-in)>
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/lfs/ampere9/0/roed/env/micromamba/envs/dogma-data/lib/python3.11/multiprocessing/spawn.py", line 122, in spawn_main
    exitcode = _main(fd, parent_sentinel)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/lfs/ampere9/0/roed/env/micromamba/envs/dogma-data/lib/python3.11/multiprocessing/spawn.py", line 132, in _main
    self = reduction.pick

ProcessExitedException: process 3 terminated with exit code 1